In [4]:
import glob
import shutil
import cv2
import os
import numpy as np


## Copy test Fovs

In [ ]:
path1 = '/opt/zhuoyao_workspace/Datasets/private_cervical/2018-12-14/FJ_1804615/Images'
path2 = '/opt/zhuoyao_workspace/Datasets/private_cervical/2018-12-14/HD_180130/Images'
path3 = '/opt/zhuoyao_workspace/Datasets/private_cervical/2018-12-14/HYN_HSIL_T219590/Images'
path4 = '/opt/zhuoyao_workspace/Datasets/private_cervical/2018-12-14/HYN_LSIL_T199784/Images'


filelist_HSIL = os.listdir(path1)
filelist_LSIL = os.listdir(path2)
filelist_SCC = os.listdir(path3)
filelist_LSIL_HPV = os.listdir(path4)
#filelist_NILM = os.listdir(path5)

filelists = [filelist_HSIL, filelist_LSIL, filelist_SCC, filelist_LSIL_HPV]
paths = [path1, path2, path3, path4]
c_lists = []
for filelist, path in zip(filelists, paths):
    index = np.random.choice(len(filelist), size=25)
    c_list = []
    for file in np.array(filelist)[index]:
        c_list.append(os.path.join(path,file))
    c_lists.append(c_list)

#prefix = ['HSIL_', 'LSIL_', 'SCC_', 'LSIL_HPV', 'NILM_H_']

prefix = ['FJ_', 'HD_', 'HYN_HSIL_', 'HYN_LSIL_']
dst_path = '/opt/zhuoyao_workspace/final_medical_cervix_2/datasets/origin_test'
for i, c_list in enumerate(c_lists):
    for file in c_list:
        write_path = os.path.join(dst_path, prefix[i] + os.path.basename(file).split('.')[0]+'.png')
        img = cv2.imread(file, 1)
        cv2.imwrite(write_path, img)

## Copy labeled file to project

In [11]:
def copy_data_to_project(src_paths, dst_path, label_paths, prefix_list):
    #assert len(src_paths) == len(dst_paths),"Src list and dst list missmatch"
    if label_paths:
        assert len(src_paths) == len(label_paths), "Src list and label_list missmatch"
    if prefix_list:
        assert len(src_paths) == len(prefix_list), "Src list and prefix list missmatch"
    
    labellists = []
    for label_path in label_paths:
        labels = os.listdir(label_path)
        labels.sort()
        labellists.append(labels)
        
    for src_path, labellist, prefix in zip(src_paths, labellists, prefix_list):
        for file_name in labellist:
            if file_name.split('.')[-1] not in ['JPG', 'PNG', 'jpg', 'png']:
                continue
            file = os.path.join(src_path, file_name)
            if os.path.exists(file):
                write_path = os.path.join(dst_path, prefix + '_' + file_name.split('.')[0]+'.png')
                img = cv2.imread(file, 1)
                cv2.imwrite(write_path, img)
            else:
                print("File did not exsit: %s" % file)
        

In [13]:
src_paths = ['/opt/zhuoyao_workspace/Datasets/private_cervical/2019-1-15/src/HSIL_1818752/Images']
dst_path = '/opt/zhuoyao_workspace/final_medical_cervix_2/datasets/origin_train/origin_01_15'
label_paths = ['/opt/zhuoyao_workspace/Datasets/private_cervical/2019-1-15/label/HSIL_1818752/']
copy_data_to_project(src_paths, dst_path, label_paths, ['HSIL'])

## Copy different fovs from same slide

In [61]:
import glob
import os
import random
import re
import cv2

data_set = '../../../../Datasets/private_cervical/origin/'
dst_dir = '../../../datasets/origin_test/2019-01-03'
prev = ['../../../datasets/origin_train/origin_12_06', 
        '../../../datasets/origin_test/2018-12-06/', 
        '../../../datasets/origin_test/2018-12-14/']

In [81]:
data_set = '../../../../Datasets/private_cervical/2018-12-14/'
dst_dir = '../../../datasets/origin_test/2019-01-04'
prev = ['../../../datasets/origin_train/origin_12_06', 
        '../../../datasets/origin_test/2018-12-06/', 
        '../../../datasets/origin_test/2018-12-14/']

In [89]:
dirs = os.listdir(data_set)

if not os.path.exists(dst_dir):
    os.makedirs(dst_dir)
if os.listdir(dst_dir):
    for file in glob.glob(dst_dir + '/*'):
        os.remove(file)

In [90]:
for folder in dirs:
    abs_folder = os.path.join(data_set, folder, 'Images')
    if os.path.isdir(abs_folder):
        files = os.listdir(abs_folder)
        splits = re.split('[()（）,_]', folder)
        if len(splits) > 2:
            prefix = splits[1] if not splits[2] in ['HPV', "HSIL", "LSIL"] else splits[1] + '_' + splits[2]
        else:
            prefix = splits[1]
        index = random.sample(range( len(files) ), 20)
        files = list(map(lambda x: files[x],index))
        for file in files:
            for prev_folder in prev:
                if not file in os.listdir(prev_folder):
                    src = os.path.join(abs_folder,file)
                    dst = os.path.join(dst_dir,prefix + '_' + file[:9] + '.png')
                    cv2.imwrite(dst, cv2.imread(src))
                    
            
            

In [72]:
for folder in prev:
    for file in os.listdir(dst_dir):
        if file in os.listdir(folder):
            print(file)

In [80]:
len(os.listdir(dst_dir))

60